In [1]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.tools import tool
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
import yfinance as yf
from langchain_core.messages import HumanMessage
import re

@tool
def stock_news(ticker: str) -> list:
    """
    Retrieve the latest news articles discussing a particular stock ticker.
    Parameters:
        - ticker: Stock ticker symbol (e.g., AAPL for Apple).
    Returns:
        - A list of dictionaries containing limited news articles.
    """
    max_articles=2
    # Get the ticker object
    ticker_obj = yf.Ticker(ticker)
    news = ticker_obj.get_news()  # Fetch news articles

    # Ensure `news` is a list and limit the articles to max_articles
    if isinstance(news, list):
        selected_news = news[:max_articles]  # Limit the articles
        
        # Process and return formatted articles
        formatted_news = []
        for article in selected_news:
            article=article.get("content")
            formatted_article = {
                "Summary": article.get("summary", "Summary not available"),
            }
            formatted_news.append(formatted_article)

        return formatted_news

    return []

tools=[stock_news]
llm = ChatOllama(
    model="llama3.1",
    temperature=0.5,
    max_tokens=1024
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a financial assistant specializing in risk assessment. Analyze the news content provided to evaluate the risk level of investing in the mentioned stock. Use the following categories for classification: 'Low Risk,' 'Medium Risk,' or 'Very Risky.' You do not need to know the stock price to make this assessment. Provide a concise explanation of your reasoning along with the risk classification. Summarize the answer.",
        ),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools,handle_parsing_errors=True)

import pandas as pd
# "AAPL", "MSFT", "GOOGL", "AMZN", "TSLA"
tickers = ["AAPL"]
responses = {}

# Iterate through each ticker
for ticker in tickers:
    # Construct the query for each ticker
    query = f"Should I buy {ticker} shares this week?"
    
    # Call the agent executor for each query
    resp = agent_executor.invoke({"messages": [HumanMessage(query)]})
    
    # Extract the content of the response, assuming it's in the format you mentioned
    response_content = resp["output"]
    
    # Store the response content in the dictionary with ticker as the key
    responses[ticker] = response_content


# Convert the responses dictionary to a DataFrame
risks = pd.DataFrame(list(responses.items()), columns=["Ticker", "Response"])
risk_pattern = r"(Low Risk|Medium Risk|Very Risky)"

# Apply the regex to extract the risk classification and store it in a new column
risks['Risk Classification'] = risks['Response'].apply(lambda x: re.search(risk_pattern, x).group(0) if re.search(risk_pattern, x) else "Not Classified")


In [2]:
risks.head()

,Ticker,Response,Risk Classification
0,AAPL,"Based on the news content provided, I would cl...",Medium Risk


In [6]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.tools import tool
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks import StreamingStdOutCallbackHandler
import yfinance as yf
from langchain_core.messages import HumanMessage
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

@tool
def stock_news(ticker: str) -> list:
    """
    Retrieve the latest news articles discussing a particular stock ticker.
    Parameters:
        - ticker: Stock ticker symbol (e.g., AAPL for Apple).
    Returns:
        - A list of dictionaries containing limited news articles.
    """
    max_articles=2
    # Get the ticker object
    ticker_obj = yf.Ticker(ticker)
    news = ticker_obj.get_news()  # Fetch news articles

    # Ensure `news` is a list and limit the articles to max_articles
    if isinstance(news, list):
        selected_news = news[:max_articles]  # Limit the articles
        
        # Process and return formatted articles
        formatted_news = []
        for article in selected_news:
            article=article.get("content")
            formatted_article = {
                "Summary": article.get("summary", "Summary not available"),
            }
            formatted_news.append(formatted_article)

        return formatted_news

    return []

tools=[stock_news]
llm = ChatOllama(
    model="llama3.1",
    temperature=0.5,
    callback_manager=callback_manager,
    verbose=True,
    show_progress=True,
    max_tokens=1024
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a financial assistant specializing in risk assessment. Analyze the news content provided to evaluate the risk level of investing in the mentioned stock. Use the following categories for classification: 'Low Risk,' 'Medium Risk,' or 'Very Risky.' You do not need to know the stock price to make this assessment. Provide a concise explanation of your reasoning along with the risk classification. Summarize the answer.",
        ),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools,handle_parsing_errors=True, verbose=True)

In [7]:
from langchain_core.messages import HumanMessage

resp = agent_executor.invoke({"messages": [HumanMessage(content="What happened to Microsoft stock prices today?")]})



> Entering new AgentExecutor chain...

Invoking: `stock_news` with `{'ticker': 'MSFT'}`


[{'Summary': "Market Domination Overtime hosts Julie Hyman&nbsp;and Josh Lipton detail the top market events on the horizon for the upcoming week. A slew of Big Tech earnings is scheduled for release, with industry giants Tesla (TSLA), Microsoft (MSFT), Meta Platforms (META), and Apple (AAPL) preparing to unveil their quarterly results. Artificial intelligence will be a central theme in these earnings reports. Additionally, the Federal Reserve's January Federal Open Market Committee (FOMC) meeting will commence on Tuesday, with the central bank's interest rate decision set to be announced on Wednesday at 2 p.m. ET. To watch more expert insights and analysis on the latest market action, check out more Market Domination Overtime&nbsp;here. This post was written by Angel Smith"}, {'Summary': 'HP CEO Enrique Lores'}]Based on the news content provided, there is no specific information about Microsoft

In [8]:
resp = agent_executor.invoke({"messages": [HumanMessage(content="Should I buy Apple shares this week?")]})



> Entering new AgentExecutor chain...

Invoking: `stock_news` with `{'ticker': 'AAPL'}`


[{'Summary': "Market Domination Overtime hosts Julie Hyman&nbsp;and Josh Lipton detail the top market events on the horizon for the upcoming week. A slew of Big Tech earnings is scheduled for release, with industry giants Tesla (TSLA), Microsoft (MSFT), Meta Platforms (META), and Apple (AAPL) preparing to unveil their quarterly results. Artificial intelligence will be a central theme in these earnings reports. Additionally, the Federal Reserve's January Federal Open Market Committee (FOMC) meeting will commence on Tuesday, with the central bank's interest rate decision set to be announced on Wednesday at 2 p.m. ET. To watch more expert insights and analysis on the latest market action, check out more Market Domination Overtime&nbsp;here. This post was written by Angel Smith"}, {'Summary': "Wealth anchor Brad Smith highlights President Trump’s initiative to roll back diversity, equity, and inclus

In [9]:
import pandas as pd

tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA"]
responses = {}

# Iterate through each ticker
for ticker in tickers:
    # Construct the query for each ticker
    query = f"Should I buy {ticker} shares this week?"
    
    # Call the agent executor for each query
    resp = agent_executor.invoke({"messages": [HumanMessage(content=query)]})
    
    # Store the response in the dictionary with ticker as the key
    responses[ticker] = resp

# Convert the responses dictionary to a DataFrame
risks = pd.DataFrame(list(responses.items()), columns=["Ticker", "Response"])



> Entering new AgentExecutor chain...

Invoking: `stock_news` with `{'ticker': 'AAPL'}`


[{'Summary': "Market Domination Overtime hosts Julie Hyman&nbsp;and Josh Lipton detail the top market events on the horizon for the upcoming week. A slew of Big Tech earnings is scheduled for release, with industry giants Tesla (TSLA), Microsoft (MSFT), Meta Platforms (META), and Apple (AAPL) preparing to unveil their quarterly results. Artificial intelligence will be a central theme in these earnings reports. Additionally, the Federal Reserve's January Federal Open Market Committee (FOMC) meeting will commence on Tuesday, with the central bank's interest rate decision set to be announced on Wednesday at 2 p.m. ET. To watch more expert insights and analysis on the latest market action, check out more Market Domination Overtime&nbsp;here. This post was written by Angel Smith"}, {'Summary': "Wealth anchor Brad Smith highlights President Trump’s initiative to roll back diversity, equity, and inclus

In [6]:
import pandas as pd
import re
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.tools import tool
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage
import yfinance as yf

@tool
def stock_news(ticker: str) -> list:
    """
    Retrieve the latest news articles discussing a particular stock ticker.
    Parameters:
        - ticker: Stock ticker symbol (e.g., AAPL for Apple).
    Returns:
        - A list of dictionaries containing limited news articles.
    """
    max_articles = 2
    # Get the ticker object
    ticker_obj = yf.Ticker(ticker)
    news = ticker_obj.get_news()  # Fetch news articles

    # Ensure `news` is a list and limit the articles to max_articles
    if isinstance(news, list):
        selected_news = news[:max_articles]  # Limit the articles
        
        # Process and return formatted articles
        formatted_news = []
        for article in selected_news:
            article = article.get("content")
            formatted_article = {
                "Summary": article.get("summary", "Summary not available"),
            }
            formatted_news.append(formatted_article)

        return formatted_news

    return []

# Define tickers corresponding to the companies
tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA"]

tools = [stock_news]
llm = ChatOllama(
    model="llama3.1",
    temperature=0.5,
    max_tokens=1024,
    show_progress=True,
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a financial assistant specializing in risk assessment.
            Analyze the news content provided to evaluate the risk level of investing in the mentioned stock.
            Use the following categories for classification: 'Low Risk,' 'Medium Risk,' or 'Very Risky'.
             Provide a concise explanation of your reasoning along with the risk classification.
             Summarize the answer and use always stock_news tool.""",
        ),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True,verbose=True)

responses = {}

# Iterate through each ticker
for ticker in tickers:
    # Construct the query for each ticker
    query = f"Should I buy {ticker} shares this week?"
    
    # Call the agent executor for each query
    resp = agent_executor.invoke({"messages": [HumanMessage(content=query)]})
    
    # Extract the content of the response
    response_content = resp["output"]
    
    # Store the response content in the dictionary with ticker as the key
    responses[ticker] = response_content

# Convert the responses dictionary to a DataFrame
risks = pd.DataFrame(list(responses.items()), columns=["Ticker", "Response"])

# Regex pattern to extract risk classification (Low Risk, Medium Risk, or Very Risky)
risk_pattern = r"(Low Risk|Medium Risk|Very Risky)"

# Apply the regex to extract the risk classification and store it in a new column
risks['Risk Classification'] = risks['Response'].apply(lambda x: re.search(risk_pattern, x).group(0) if re.search(risk_pattern, x) else "Not Classified")




> Entering new AgentExecutor chain...

Invoking: `stock_news` with `{'ticker': 'AAPL'}`


[{'Summary': 'Apple released new software updates on Monday, including iOS 18.3, which automatically opts users into Apple Intelligence, at least for newer devices. Unlike what the tech industry seems to think, or wants to believe, not everyone wants generative AI features enabled by default on their devices. You may want to switch off the new AI features because you are concerned about the AI industry’s impact on the environment or some of the political views of the industry’s top players, or you just don’t'}, {'Summary': 'Apple stock jumped Monday, proving itself resilient to the widespread AI stock sell-off Monday prompted by the release of Chinese firm DeepSeek’s new cost-effective artificial intelligence model.'}]Based on the news content provided, I would classify the risk level of investing in AAPL shares as 'Low Risk'. The reason for this classification is that despite the recent AI-rela

In [10]:
risks

,Ticker,Response,Risk Classification
0,AAPL,"Based on the news content provided, I would cl...",Low Risk
1,MSFT,"Based on the news content provided, I would cl...",Medium Risk
2,GOOGL,"Based on the provided news content, I would cl...",Medium Risk
3,AMZN,"Based on the provided news content, I would cl...",Medium Risk
4,TSLA,"Based on the provided news content, I would cl...",Medium Risk
